In [1]:
import numpy as np
import time


def soft_thresholding(x, a):
    # assuming a is non-negative
    a = np.array(a)
    x = np.array(x)
    result = np.zeros(x.size)
    result[x > a] = x[x > a] - a
    result[x < -a] = x[x < -a] + a
    return result


def lasso_kkt_check(X, y, beta, lambda_val, tol=1e-3):
    # check convergence
    G = X.T.dot(y - X.dot(beta)) / y.size
    ix = beta == 0
    iy = beta != 0
    if np.any(np.abs(G[ix]) > (lambda_val + tol)):
        return False
    if np.any(np.abs(G[iy] - lambda_val * np.sign(beta[iy])) > tol):
        return False
    return True


def lasso_cd(X, y, beta, lambda_val, tol=1e-6, max_iter=1000, quiet=False):
    # note that the LS part  in this function is the one in slides divided by length(y) = n
    # Or equivalently  lambda here = n * lambda in class

    obj = np.zeros(max_iter + 1)
    beta_list = [np.copy(beta)]

    j = 0
    for j in range(max_iter):
        for k in range(beta.size):
            r = y - np.delete(X, k, axis=1).dot(np.delete(beta, k))

            beta[k] = (1 / (np.linalg.norm(X[:, k]) ** 2)) \
                      * soft_thresholding([r.T.dot(X[:, k])], [len(y) * lambda_val])
        beta_list.append(np.copy(beta))
        obj[j] = (1/2) * (1/len(y)) * np.linalg.norm(y - X.dot(beta)) ** 2 + lambda_val * np.sum(np.abs(beta))

        if np.linalg.norm(beta_list[j] - beta) < tol:
            break

    check = lasso_kkt_check(X, y, beta, lambda_val)
    if not quiet:
        if check:
            print("Minimum obtained")
        else:
            print("Minimum not obtained")

    return {
        'obj': obj[:j],
        'beta': beta
    }


In [2]:
# Model
n = 50
p = 400

X = np.random.normal(size=(n, p))
b = np.zeros(400)

b[300:305] = np.arange(10, 0, -2)

y = np.dot(X, b) + np.random.normal(size=(n,))
y_new = np.dot(X, b) + np.random.normal(size=(n,))

In [3]:
start_time = time.time()
re = lasso_cd(X, y, np.zeros(400), .1)
end_time = time.time()
print(end_time - start_time)

Minimum obtained
7.859547138214111


In [6]:
re['beta'][295:310], b[295:310]

(array([0.        , 0.        , 0.        , 0.        , 0.        ,
        9.32543655, 7.67390439, 6.05526821, 3.88956063, 1.48348612,
        0.        , 0.        , 0.        , 0.        , 0.        ]),
 array([ 0.,  0.,  0.,  0.,  0., 10.,  8.,  6.,  4.,  2.,  0.,  0.,  0.,
         0.,  0.]))

In [19]:
start_time = time.time()
re = lasso_cd(X, y, np.zeros(400), 1)
end_time = time.time()
print(end_time - start_time)

Minimum obtained
0.806614875793457


In [20]:
# warm start
start_time = time.time()
lasso_cd(X, y, re['beta'], 0.1)
end_time = time.time()
print(end_time - start_time)

Minimum obtained
3.9056928157806396
